In [ ]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu unstructured unstructured[pdf]

In [7]:
!pip freeze > requirements.txt

In [1]:
!which python
!python --version

/Users/desi76/repo-git-nugh/doc-qa/mye/bin/python
Python 3.9.19


In [2]:
import os

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA


In [3]:
from dotenv import load_dotenv
import os

# Carica le variabili di ambiente dal file .env
load_dotenv()

# Ora puoi accedere alle variabili di ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
#llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)


In [5]:
# load the document
path_file="/Users/desi76/repo-git-nugh/doc-qa/pdfs/hoder.pdf"
loader = UnstructuredFileLoader(path_file)
documents = loader.load()

/Users/desi76/repo-git-nugh/doc-qa/mye/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
type(documents[0])

langchain_core.documents.base.Document

In [7]:
# create text chunks

text_splitter = CharacterTextSplitter(separator='/n',
                                      chunk_size=2000,
                                      chunk_overlap=200)

text_chunks = text_splitter.split_documents(documents)

Created a chunk of size 32881, which is longer than the specified 2000


In [8]:
# Inizializza HuggingFaceEmbeddings con il modello desiderato
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Sostituisci con il modello Hugging Face desiderato
embeddings = HuggingFaceEmbeddings(model_name=model_name)
# vector embedding for text chunks
knowledge_base = FAISS.from_documents(text_chunks, embeddings)
# Stampa alcune informazioni sulla knowledge base
print("Knowledge Base creata con successo.")

Knowledge Base creata con successo.


In [9]:
# chain for aq retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [10]:

# List of questions
questions = [
    "Qual è il background dell'articolo",
    "Quali sono gli obiettivi",
    "Qual è la metodologia",
    "Quali sono gli strumenti di rilevamento dei dati",
    "Quali sono i risultati",
    "Qual è la discussione",
    "Quali sono le conclusioni"
]

# Initialize an empty list to store the results
results = []

# Iterate through the list of questions and get the responses
for question in questions:
    response = qa_chain.invoke({"query": question})
    results.append(response["result"])

# Define the output file path
output_file = "results.txt"

# Write the results to a text file
with open(output_file, "w") as file:
    for i, result in enumerate(results):
        file.write(f"Question {i + 1}:\n{questions[i]}\n")
        file.write(f"Response {i + 1}:\n{result}\n\n")

print(f"Results have been written to {output_file}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Results have been written to results.txt


In [11]:
print(results)

["L'articolo si concentra sull'approccio diasistemico alle costruzioni multilingui. Esso esplora come i parlanti multilingui creino un sistema comune per tutte le loro lingue, incorporando sia collegamenti interlinguistici che elementi non specifici della lingua insieme a strutture specifiche della lingua. L'autore si basa su un'analisi della grammatica delle costruzioni per concettualizzare questi sistemi e i cambiamenti al loro interno. L'articolo si concentra su casi specifici di contatto linguistico, come ad esempio il contatto tra il latino medievale e il svedese antico, per illustrare come avvengono i cambiamenti e la stabilità in situazioni di contatto linguistico.", "Gli obiettivi principali di questo studio sono:\n\n1. Illustrare come i parlanti multilingue identificano e collegano categorie grammaticali tra lingue diverse, creando una rete di costruzioni interlinguali.\n   \n2. Mostrare come la creazione di una rete di costruzioni interlinguali faciliti la comprensione e l'us